# Исследование надёжности заёмщиков

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

Согласно документации к данным:  
- `children` — количество детей в семье  
- `days_employed` — общий трудовой стаж в днях  
- `dob_years` — возраст клиента в годах  
- `education` — уровень образования клиента  
- `education_id` — идентификатор уровня образования  
- `family_status` — семейное положение  
- `family_status_id` — идентификатор семейного положения  
- `gender` — пол клиента  
- `income_type` — тип занятости  
- `debt` — имел ли задолженность по возврату кредитов  
- `total_income` — ежемесячный доход  
- `purpose` — цель получения кредита  

## Шаг 1. Загрузка данных и изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
import numpy as np

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод**

- В таблице двенадцать столбцов. Тип данных в столбцах - `int64`, `float64`, `object`.
- В названиях столбцов нарушений нет.
- Каждая строка таблицы-данные о заемщике.
- В столбцах `days_employed` и `total_income` встречаются пропуски.  
- В столбце `days_employed` встречаются отрицательные и некорректные значения.  
- В столбце `education` встречаются неявные дубликаты.

## Шаг 2. Предобработка данных

### Обработка пропусков

Определим количество пропусков

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
data[data['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пропуски в столбце `total_income` заполним медианным значением с учетом типа дохода, семейного положения и образования.

In [8]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'family_status_id', 
                                                                 'education_id'])['total_income'].transform('median'))
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           2
purpose                0
dtype: int64

Отавшиеся пропуски в столбце `total_income` заполним медианным значением с учетом типа дохода и образования.

In [9]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'education_id']
                                                               )['total_income'].transform('median'))
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

В столбце `days_employed` уберем отрицательные значения.

In [10]:
data['days_employed'] = data['days_employed'].abs()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Если предположить что аномально большие значения, это стаж в часах, перезапишем их, разделив на 24.

In [11]:
data.loc[data['days_employed'] > 25000, 'days_employed'] = data.loc[data['days_employed'] > 25000, 'days_employed'] / 24
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Пропуски в столбце `days_employed` заполним средним значением с учетом возраста и типа занятости.

Для этого проверим столбец `dob_years`.

In [12]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце `dob_years` обнаружены аномальные значения возраста равные 0, их доля около одного процента. Для того чтобы не учитывать при заполнении столбца `days_employed`, заменим их на NaN.

In [13]:
data['dob_years'].replace(0, np.NaN, inplace = True)
data.isna().sum()

children               0
days_employed       2174
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Заполним пропуски в столбце `days_employed`.

In [14]:
data['days_employed'] = data['days_employed'].fillna(data.groupby(['dob_years', 'income_type']
                                                                 )['days_employed'].transform('mean'))
data.isna().sum()

children              0
days_employed        14
dob_years           101
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
dtype: int64

Оставшиеся пропуски заполним средним значением с учетом типа занятости.

In [15]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('mean'))
data.isna().sum()

children              0
days_employed         0
dob_years           101
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
dtype: int64

Проверим столбец `children`

In [16]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значения -1 и 20 скорее всего являются значениями по умолчанию, применяемыми из-за отсутствия информации, их доля невелика, около одного процента. Поэтому заменим их на NaN, для того что бы они не искажали дальнейшие расчеты.

In [17]:
data['children'].replace(-1, np.NaN, inplace = True)
data['children'].replace(20, np.NaN, inplace = True)
data['children'].value_counts()

0.0    14149
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

**Вывод**
- Пропуски заполнены средними и медианными значениями в зависимости от других характеристик
- Возможная причина появления пропусков-сбой в системе сбора информации.

### Замена типов данных

Заменим вещественный тип данных в столбцах `total_income` и `days_employed`, на целочисленный, для удобства дальнейших расчетов.

In [18]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**
- Вещественный тип данных, скорее всего возникает в процессе расчетов.

### Обработка дубликатов

Проверим наличие явных дубликатов:

In [19]:
data.duplicated().sum()
print('Количество дубликатов: {}'.format(data.duplicated().sum()))

Количество дубликатов: 54


Для удаления явных дубликатов сначала избавимся от неявных дубликатов.

Обработаем столбец `education`.

In [20]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Приведем значения в столбце `education` к нижнему регистру.

In [21]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [22]:
data.duplicated().sum()
print('Количество дубликатов: {}'.format(data.duplicated().sum()))

Количество дубликатов: 71


Проверим столбец `family_status`.

In [23]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Приведем значения в столбце `family_status` к нижнему регистру.

In [24]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Проверим столбец `income_type`.

In [25]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Посчитаем и удалим явные дубликаты.

In [26]:
data.duplicated().sum()
print('Количество дубликатов: {}'.format(data.duplicated().sum()))

Количество дубликатов: 71


In [27]:
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов: {}'.format(data.duplicated().sum()))

Количество дубликатов: 0


**Вывод**
- Были выявлены и удалены явные и неявные дубликаты. Причины возникновения-человеческий фактор и технические ошибки.

### Лемматизация

Цели кредитов, в столбце `purpose`, записаны различными формулировками, поэтому просмотрим все уникальные значения, выделим леммы и разделим цели по категориям, для этого добавим столбец `purpose_category`.

In [28]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Исходя из списка целей, выделим четыре основные категории:  
- недвижимость  
- автомобиль  
- образование  
- свадьба

In [29]:
def purpose_lemmas(row):
    lemmas_row = m.lemmatize(row)
    if 'автомобиль' in lemmas_row:
        return 'автомобиль'
    if 'недвижимость' in lemmas_row:
        return  'недвижимость'
    if 'жилье' in lemmas_row:
        return  'недвижимость'
    if 'свадьба' in lemmas_row:
        return  'свадьба'
    if 'образование' in lemmas_row:
        return 'образование'
    
data['purpose_category']= data['purpose'].apply(purpose_lemmas)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0.0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0.0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0.0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2.0,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0.0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [30]:
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

**Вывод**

Разделили цели на четыре категории:  
- недвижимость - все, что связано с недвижимостью  
- автомобиль - покупка автомобиля  
- образование - получение образования  
- свадьба - проведение свадьбы

### Категоризация данных

Выделим категории по наличию детей:
- 0 - нет детей
- 1 - есть дети

Категории добавим в столбец `children_category`.

In [31]:
def total_children_category(children):
    if children == 0:
        return 0
    if children > 0:
        return 1
    
data['children_category'] = data['children'].apply(total_children_category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1.0
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1.0
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0.0
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1.0
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0.0


Для дальнейших расчетов, необходимо уровень ежемесячного дохода заемщиков разделить на категории.

- 0 - от 0 до 50000  
- 1 - от 50001 до 100000  
- 2 - от 100001 до 150000  
- 3 - от 150001 до 200000  
- 4 - от 200001 и выше

Категории дохода добавим в столбец `income_category`.

In [32]:
def total_income_category(income):
    if income <= 50000:
        return 0
    if income <= 100000:
        return 1
    if income <= 150000:
        return 2
    if income <= 200000:
        return 3
    if income > 200000:
        return 4

data['income_category'] = data['total_income'].apply(total_income_category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_category
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1.0,4
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1.0,2
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0.0,2
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1.0,4
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0.0,3


Выделим словарь по категориям семейного положения, т. к. он понадобится для дальнейших расчетов.

In [33]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


Удалим дубликаты.

In [34]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


**Вывод**

Категории ежемесячного дохода (стобец`income_category`):

- 0 - от 0 до 50000  
- 1 - от 50001 до 100000  
- 2 - от 100001 до 150000  
- 3 - от 150001 до 200000  
- 4 - от 200001 и выше

Категории семейного положения (столбец `family_status_id`):

- 0 - женат / замужем
- 1 - гражданский брак
- 2 - вдовец / вдова
- 3 - в разводе
- 4 - не женат / не замужем

Категории по наличию детей (столбец `children_category`):  
 - 0 - нет детей
 - 1 - есть дети

## Шаг 3. Ответы на вопросы

 ### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [35]:
def conversion(column):
    conv_table = data.pivot_table(index = column, values = 'debt', aggfunc = ['sum', 'count', 'mean'])
    conv_table.columns = ['Всего должников', 'Всего кредитополучателей', 'Доля должников']
    conv_table = conv_table.sort_values(by = ['Доля должников'], ascending=False)
    return(conv_table.style.format({'Доля должников': '{:.2%}'}))

In [36]:
conversion('children')

,Всего должников,Всего кредитополучателей,Доля должников
children,,,
4.0,4,41,9.76%
2.0,194,2052,9.45%
1.0,444,4808,9.23%
3.0,27,330,8.18%
0.0,1063,14091,7.54%
5.0,0,9,0.00%


**Вывод:**

Из данной таблицы видно, что процент не выплаты кредита в срок больше у клиентов с детьми. И подавляющее большинство просрочек в семьях которых от 1 до 2 детей, но это связано с тем, что таких семей в принципе больше, чем семей с 3 и более детьми.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
conversion('family_status')

,Всего должников,Всего кредитополучателей,Доля должников
family_status,,,
не женат / не замужем,274,2810,9.75%
гражданский брак,388,4151,9.35%
женат / замужем,931,12339,7.55%
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%


**Вывод:**

Из данной таблицы видно, что самые дисциплинированные плательщики женатые либо кто был ранее женат/замужем. Возможно по причине большей ответственности перед семьей

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
income_category = '''Категории по уровню дохода:
0 - от 0 до 50000
1 - от 50001 до 100000
2 - от 100001 до 150000
3 - от 150001 до 200000
4 - от 200001 и выше'''
print(income_category)

conversion('income_category')

Категории по уровню дохода:
0 - от 0 до 50000
1 - от 50001 до 100000
2 - от 100001 до 150000
3 - от 150001 до 200000
4 - от 200001 и выше


,Всего должников,Всего кредитополучателей,Доля должников
income_category,,,
2,615,6995,8.79%
3,409,4831,8.47%
1,331,4092,8.09%
4,363,5164,7.03%
0,23,372,6.18%


**Вывод:** 

Основная масса заемщиков из категории "2" - это люди со средним доходом по РФ, но так же и одна из самых безответственных в плане выплаты кредита в срок. Категория "4" не задерживает выплаты по кредиту, т.к. их доход выше среднего и думаю они более подкованные в финансовом плане и понимают, чем чревато невыплата кредита в срок.

### Как разные цели кредита влияют на его возврат в срок?

In [41]:
conversion('purpose_category')

,Всего должников,Всего кредитополучателей,Доля должников
purpose_category,,,
автомобиль,403,4306,9.36%
образование,370,4013,9.22%
свадьба,186,2324,8.00%
недвижимость,782,10811,7.23%


**Вывод:**

Покупка недвижимости серьезный шаг, поэтому и подходят к нему осознано и с пониманием, что при несвоевременной оплате можно потерять жилье. А вот покупка автомобиля может быть спонтанна и так же не в соответствии с уровнем дохода.

## Шаг 4. Общий вывод

 Самые привлекательные клиенты для выдачи им кредита это женатые/замужние люди, без детей, берущие деньги на недвижимость и с доходом от 200001 рублей. 
 А вот одни из самых рискованных клиентов это не женатые/не замужние люди, с 1 или 2 детьми, оформляющие кредит на автомобиль с доходом от 50000 до 200000 рублей